In [ ]:
#관련 라이브러리 설치

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install google-generativeai datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7zc51cni/unsloth_775e780b6de14c29971aa1d38136bdf6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7zc51cni/unsloth_775e780b6de14c29971aa1d38136bdf6
  Resolved https://github.com/unslothai/unsloth.git to commit 2eb6b0d5f363a60ed3792ea1f04250537ac66939
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 136.3 MB/s eta 0:00:00

# 데이터 및 모델 설정

In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from collections import defaultdict
from tqdm import tqdm

max_seq_length = 2048
dtype = None
load_in_4bit = True

model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
dataset_name = "Blpeng/nsmc"

# 데이터 로드 & 샘플링
dataset = load_dataset(dataset_name)
sampled_train = dataset["train"].shuffle(3407).select(range(1000))
sampled_test  = dataset["test" ].shuffle(3407).select(range(1000))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


README.md:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


ratings_train.csv:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

ratings_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)
print("train columns:", dataset["train"].column_names)
print("test columns:", dataset["test"].column_names)
print("train sample:", dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 50000
    })
})
train columns: ['Unnamed: 0', 'id', 'document', 'label']
test columns: ['Unnamed: 0', 'id', 'document', 'label']
train sample: {'Unnamed: 0': 0, 'id': 9976970, 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}


# 한국어 프롬프트로 변경

In [ ]:
# 프롬프트/라벨
prompt = """다음은 영화 리뷰입니다:
{}

이 리뷰의 감성은 무엇인가요? "긍정" 또는 "부정"으로만 답하세요.

정답
정답은: "{}"""

positivelabel = "긍정"
negativelabel = "부정"

In [ ]:
def formatting_prompts_func(dataset_):
    if isinstance(dataset_["document"], str):
        t = dataset_["document"]
        y = dataset_["label"]
        label = positivelabel if y == 1 else negativelabel
        return [prompt.format(t, label)]

    texts = []
    for t, y in zip(dataset_["document"], dataset_["label"]):
        label = positivelabel if y == 1 else negativelabel
        texts.append(prompt.format(t, label))
    return texts

In [ ]:
# 평가 함수
def test_model(model, tokenizer, test_dataset, pos_token_id, neg_token_id):
    tokenized_inputs = []
    for i in tqdm(range(len(test_dataset["document"]))):
        text = test_dataset["document"][i]
        test_str = prompt.format(text, "")
        tokenized_input = tokenizer(test_str, return_tensors="pt", add_special_tokens=False)
        tokenized_inputs.append((tokenized_input, test_dataset["label"][i]))

    tokenized_inputs.sort(key=lambda x: x[0]["input_ids"].shape[1])

    grouped_inputs = defaultdict(list)
    for tokenized_input, label in tokenized_inputs:
        length = tokenized_input["input_ids"].shape[1]
        grouped_inputs[length].append((tokenized_input, label))

    batch_size = 32
    all_outputs, all_labels = [], []

    for length, group in tqdm(grouped_inputs.items()):
        for i in range(0, len(group), batch_size):
            batch = group[i:i+batch_size]
            batch_inputs = [item[0] for item in batch]
            batch_labels = [item[1] for item in batch]

            input_ids = torch.cat([item["input_ids"] for item in batch_inputs], dim=0).to("cuda")
            attention_mask = torch.cat([item["attention_mask"] for item in batch_inputs], dim=0).to("cuda")

            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            last_token_logits = outputs.logits[:, -1, :]  # 마지막 토큰
            neg_pos_logits = last_token_logits[:, [neg_token_id, pos_token_id]]  # [neg, pos]
            probs = torch.softmax(neg_pos_logits, dim=-1)
            preds = torch.argmax(probs, dim=-1)  # 0=neg, 1=pos

            all_outputs.extend(preds.cpu().numpy().tolist())
            all_labels.extend(batch_labels)

    correct = sum(int(p) == int(y) for p, y in zip(all_outputs, all_labels))
    total = len(all_outputs)
    acc = correct / total
    print(f"Correct: {correct} Total: {total} Accuracy: {acc:.4f}")
    return acc

# BASELINE (no fine-tuning)

In [ ]:
# Baseline: LoRA 없이 원본 모델 로드
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=load_in_4bit,
    max_seq_length=max_seq_length,
    dtype=dtype,
)

# 긍정/부정 토큰 id
pos_token_id = tokenizer.encode("긍정", add_special_tokens=False)[0]
neg_token_id = tokenizer.encode("부정", add_special_tokens=False)[0]

print("=== Baseline (no fine-tuning) ===")
baseline_acc = test_model(base_model, tokenizer, sampled_test, pos_token_id, neg_token_id)

==((====))==  Unsloth 2025.12.8: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Bllossom/llama-3.2-Korean-Bllossom-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
=== Baseline (no fine-tuning) ===


100%|██████████| 100/100 [00:13<00:00,  7.20it/s]

Correct: 698 Total: 1000 Accuracy: 0.6980


# LoRA FINE-TUNING (r=4)

In [ ]:
# r=4: base model 새로 로드
model_r4, tokenizer_r4 = FastLanguageModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=load_in_4bit,
    max_seq_length=max_seq_length,
    dtype=dtype,
)


==((====))==  Unsloth 2025.12.8: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Bllossom/llama-3.2-Korean-Bllossom-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [ ]:
pos_token_id_r4 = tokenizer_r4.encode("긍정", add_special_tokens=False)[0]
neg_token_id_r4 = tokenizer_r4.encode("부정", add_special_tokens=False)[0]

In [ ]:
from peft import LoftQConfig

peft_r4 = FastLanguageModel.get_peft_model(
    model_r4,
    r=4,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.12.8 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
import torch

trainer_r4 = SFTTrainer(
    model=peft_r4,
    tokenizer=tokenizer_r4,
    train_dataset=sampled_train,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs_r4",
        num_train_epochs=1,
        report_to="none",
        group_by_length=False,
    ),
    formatting_func=formatting_prompts_func,
)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print("=== r=4 fine-tuning ===")
trainer_r4.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 63
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 1,146,880 of 3,213,896,704 (0.04% trained)


=== r=4 fine-tuning ===
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.012500
2,4.025900
3,4.123900
4,4.019600
5,4.023700
6,3.854800
7,3.815800
8,3.865900
9,3.605200
10,3.580600


TrainOutput(global_step=63, training_loss=2.436168731205047, metrics={'train_runtime': 41.1168, 'train_samples_per_second': 24.321, 'train_steps_per_second': 1.532, 'total_flos': 2092587141365760.0, 'train_loss': 2.436168731205047, 'epoch': 1.0})

In [ ]:
print("=== r=4 after fine-tuning ===")
acc_r4_after = test_model(peft_r4, tokenizer_r4, sampled_test, pos_token_id_r4, neg_token_id_r4)

=== r=4 after fine-tuning ===


100%|██████████| 100/100 [00:12<00:00,  8.01it/s]

Correct: 746 Total: 1000 Accuracy: 0.7460


# LoRA FINE-TUNING (r=2)

In [ ]:
# r=2: base model 새로 로드
model_r2, tokenizer_r2 = FastLanguageModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=load_in_4bit,
    max_seq_length=max_seq_length,
    dtype=dtype,
)

==((====))==  Unsloth 2025.12.8: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Bllossom/llama-3.2-Korean-Bllossom-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [ ]:
pos_token_id_r2 = tokenizer_r2.encode("긍정", add_special_tokens=False)[0]
neg_token_id_r2 = tokenizer_r2.encode("부정", add_special_tokens=False)[0]

In [ ]:
peft_r2 = FastLanguageModel.get_peft_model(
    model_r2,
    r=2,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True,
)

In [ ]:
import torch

trainer_r2 = SFTTrainer(
    model=peft_r2,
    tokenizer=tokenizer_r2,
    train_dataset=sampled_train,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs_r2",
        num_train_epochs=1,
        report_to="none",
        group_by_length=False,
    ),
    formatting_func=formatting_prompts_func,
)

In [ ]:
print("=== r=2 fine-tuning ===")
trainer_r2.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 63
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 573,440 of 3,213,323,264 (0.02% trained)


=== r=2 fine-tuning ===


Step,Training Loss
1,4.012500
2,4.025900
3,4.120800
4,4.023300
5,4.034600
6,3.883700
7,3.863600
8,3.942600
9,3.704800
10,3.710900


TrainOutput(global_step=63, training_loss=2.536418665023077, metrics={'train_runtime': 24.5004, 'train_samples_per_second': 40.816, 'train_steps_per_second': 2.571, 'total_flos': 2092161603010560.0, 'train_loss': 2.536418665023077, 'epoch': 1.0})

In [ ]:
print("=== r=2 after fine-tuning ===")
acc_r2_after = test_model(peft_r2, tokenizer_r2, sampled_test, pos_token_id_r2, neg_token_id_r2)

=== r=2 after fine-tuning ===


100%|██████████| 100/100 [00:12<00:00,  8.13it/s]

Correct: 721 Total: 1000 Accuracy: 0.7210


# FEW-SHOT PROMPT INFERNECE (with r=2)

In [ ]:
#메모리 정리
import gc
torch.cuda.empty_cache(); gc.collect()

798

In [ ]:
# 1) few-shot demonstration 2개 이상 포함한 프롬프트 템플릿 만들기
fewshot_header = """다음은 영화 리뷰 감성분석 예시입니다.
아래 예시를 참고하여 마지막 리뷰의 감성을 "긍정" 또는 "부정"으로만 답하세요.

[예시 1]
리뷰: 정말 감동적이고 배우들의 연기가 훌륭했어요.
감성: 긍정

[예시 2]
리뷰: 지루하고 전개가 엉망이라 끝까지 보기 힘들었어요.
감성: 부정

"""

# 2) 기존 prompt를 few-shot 형태로 교체
prompt_backup = prompt  # 원래 zero-shot prompt 백업

prompt = fewshot_header + """[문제]
리뷰: {}

감성: {}"""

print("=== LoRA(r=2) few-shot ===")
fewshot_acc = test_model(peft_r2, tokenizer_r2, sampled_test, pos_token_id_r2, neg_token_id_r2)

prompt = prompt_backup

=== LoRA(r=2) few-shot ===


100%|██████████| 98/98 [00:13<00:00,  7.47it/s]

Correct: 624 Total: 1000 Accuracy: 0.6240


# REASONING 데이터셋 생성 및 학습

In [ ]:
# 0) Imports & Logging

import google.generativeai as genai
from google.colab import userdata
import time, json, random, re, math
from tqdm import tqdm
import pandas as pd
import logging

logging.getLogger("tornado.access").setLevel(logging.ERROR)

In [ ]:
# 1) Gemini API 설정

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

model_gemini = genai.GenerativeModel("gemini-2.5-flash")

In [ ]:
# 2) 유틸: JSON 배열 추출 + "1문장" 검증

def _extract_json_array(text: str):
    text = (text or "").strip()
    text = text.replace("```json", "").replace("```", "").strip()
    start = text.find("[")
    end   = text.rfind("]")
    if start != -1 and end != -1:
        text = text[start:end+1]
    try:
        return json.loads(text)
    except Exception:
        return None


def _is_single_sentence_reason(s: str) -> bool:

    if s is None:
        return False
    s = str(s).strip()
    if not s:
        return False
    if "\n" in s or "\r" in s:
        return False

    end_ok = bool(re.search(r"[\.!?…]$", s))
    if not end_ok:
        return False

    cnt = len(re.findall(r"[\.!?…]", s))
    if cnt != 1:
        return False

    if len(s) < 6:
        return False

    return True

In [ ]:
# 3) 배치 reasoning 생성

def generate_reasoning_batch(reviews, labels, max_retries=2):

    N = len(reviews)

    prompt = f"""
너는 데이터 라벨링 전문가야.
각 리뷰의 정답 감성(긍정/부정)을 참고해서, 근거(reason)를 한국어 '딱 1문장'으로만 작성해.

반드시 아래 규칙을 지켜:
- 출력은 JSON 배열만. (설명/텍스트/코드펜스 절대 금지)
- 배열 길이는 반드시 {N}개
- 각 원소는 반드시 {{"reason":"..."}} 형태
- reason은 빈 문자열 금지
- reason은 줄바꿈 금지
- reason은 반드시 마침표(.) 또는 ! 또는 ? 또는 … 중 하나로 "끝나야" 함
- 그리고 위 종결부호는 reason 전체에서 "정확히 1번만" 등장해야 함 (두 문장 금지)
- 입력 순서 그대로 출력

출력 예시(형식만 참고):
[
  {{"reason":"감동적이고 재미있다는 표현이 있어 긍정으로 판단했다."}},
  {{"reason":"지루하고 별로라는 평가가 있어 부정으로 판단했다."}}
]
""".strip()

    for r, l in zip(reviews, labels):
        label_str = "긍정" if int(l) == 1 else "부정"
        r = str(r).replace("\n", " ").replace("\r", " ").strip()[:120]
        prompt += f'\n\n리뷰: "{r}"\n정답: {label_str}'

    max_out = min(4096, 120 * N + 200)

    gen_cfg = {
        "temperature": 0.0,
        "max_output_tokens": max_out,
        "response_mime_type": "application/json",
    }

    spent_calls = 0

    for attempt in range(max_retries):
        try:
            spent_calls += 1
            try:
                resp = model_gemini.generate_content(prompt, generation_config=gen_cfg)
            except TypeError:

                gen_cfg.pop("response_mime_type", None)
                raise

            data = _extract_json_array(resp.text)


            reasons = [None] * N

            if isinstance(data, list):

                for i in range(min(N, len(data))):
                    if isinstance(data[i], dict):
                        reason = str(data[i].get("reason", "")).strip()
                        if reason:
                            reasons[i] = reason

            raw = (resp.text or "").strip()
            for i in range(N):
                if reasons[i] is None:
                    reasons[i] = raw

            cleaned = [{"reason": r} for r in reasons]
            return cleaned, spent_calls


        except Exception as e:
            msg = str(e)

            if "429" in msg or "Too Many Requests" in msg:
                wait = min(60, 8 * (2 ** attempt)) + random.uniform(0, 2)
                print(f"[429] Backoff {wait:.1f}s")
                time.sleep(wait)
            else:
                time.sleep(1.5)

    return [], spent_calls


In [ ]:
# 4) NSMC 1000개 샘플링

from datasets import load_dataset, Dataset as HFDataset

dataset = load_dataset("Blpeng/nsmc")

SAMPLED_N = 1000
SAMPLED_TEST_N = 1000

sampled_train = dataset["train"].shuffle(seed=3407).select(range(SAMPLED_N))
sampled_test  = dataset["test"].shuffle(seed=3407).select(range(SAMPLED_TEST_N))

reviews = list(sampled_train["document"])
labels  = list(sampled_train["label"])

print("Train sampled:", len(reviews))
print("Test sampled :", len(sampled_test))


Repo card metadata block was not found. Setting CardData to empty.


Train sampled: 1000
Test sampled : 1000


In [ ]:
# 5) 비용 방지형 생성 루프

MAX_API_CALLS_TOTAL   = 600
MAX_RETRIES_PER_CALL  = 2
MAX_SPLIT_DEPTH       = 2
ALLOW_SKIP            = True

batch_size            = 20
MIN_SECONDS_PER_CALL  = 0.2

api_calls_used = 0
new_data = []
failed_items = []

pbar = tqdm(total=len(reviews), desc="Generating 1-sentence reasoning (API)")

def solve_segment(start_idx, seg_reviews, seg_labels, depth=0):

    global api_calls_used, new_data, failed_items

    if api_calls_used >= MAX_API_CALLS_TOTAL:
        return False

    results, spent = generate_reasoning_batch(seg_reviews, seg_labels, max_retries=MAX_RETRIES_PER_CALL)
    api_calls_used += spent


    if isinstance(results, list) and len(results) == len(seg_reviews):
        for j, r in enumerate(seg_reviews):
            gt = int(seg_labels[j])
            gt_str = "긍정" if gt == 1 else "부정"
            reason = str(results[j]["reason"]).strip()


            new_data.append({"review": r, "label": gt_str, "reason": reason})



        pbar.update(len(seg_reviews))
        time.sleep(MIN_SECONDS_PER_CALL)
        return True


    if depth < MAX_SPLIT_DEPTH and len(seg_reviews) > 1:
        mid = len(seg_reviews) // 2
        left_ok  = solve_segment(start_idx, seg_reviews[:mid], seg_labels[:mid], depth+1)
        right_ok = solve_segment(start_idx+mid, seg_reviews[mid:], seg_labels[mid:], depth+1)
        return left_ok and right_ok


    if ALLOW_SKIP:
        for j in range(len(seg_reviews)):
            failed_items.append((start_idx + j, int(seg_labels[j]), "final_fail"))
        pbar.update(len(seg_reviews))
        return True
    else:
        return False

i = 0
while i < len(reviews):
    if api_calls_used >= MAX_API_CALLS_TOTAL:
        print(f"\n[STOP] cost cap reached: api_calls_used={api_calls_used}")
        break

    bs = min(batch_size, len(reviews) - i)
    seg_r = reviews[i:i+bs]
    seg_l = labels[i:i+bs]

    ok = solve_segment(i, seg_r, seg_l, depth=0)
    if ok:
        i += bs
    else:
        print(f"\n[STOP] hard fail at i={i}, api_calls_used={api_calls_used}")
        break

pbar.close()

df_cot = pd.DataFrame(new_data)

print("\n=== Generation Summary ===")
print("api_calls_used:", api_calls_used)
print("generated rows:", len(df_cot), "/", len(reviews))
print("failed items :", len(failed_items))
empty_ratio = (df_cot["reason"].fillna("").str.strip() == "").mean() if len(df_cot) else 1.0
print("EMPTY reason ratio:", empty_ratio)


Generating 1-sentence reasoning (API): 100%|██████████| 1000/1000 [10:32<00:00,  1.58it/s]


=== Generation Summary ===
api_calls_used: 50
generated rows: 1000 / 1000
failed items : 0
EMPTY reason ratio: 0.0


In [ ]:
# 6) SFT용 데이터 구성 : 문제-정답-근거

reasoning_prompt_train = """문제: {review}
정답: {label}
근거: {reason}"""

def formatting_prompts_reasoning(batch):
    if isinstance(batch["review"], str):
        return [reasoning_prompt_train.format(
            review=batch["review"],
            label=batch["label"],
            reason=batch["reason"],
        )]
    return [
        reasoning_prompt_train.format(review=r, label=l, reason=rs)
        for r, l, rs in zip(batch["review"], batch["label"], batch["reason"])
    ]

train_reason_ds = HFDataset.from_pandas(df_cot.reset_index(drop=True))

print("\n=== train_reason_ds sample ===")
print(train_reason_ds[0])
print("\n=== SFT input sample ===")
print(reasoning_prompt_train.format(
    review=train_reason_ds[0]["review"],
    label=train_reason_ds[0]["label"],
    reason=train_reason_ds[0]["reason"],
))



=== train_reason_ds sample ===
{'review': '저는 오히려 결말이 좋았다고 느꼈는데 보여주진 않았지만 저는 만났을 거라고 생각합니다. 잔잔한 여운과 쓸쓸함이 느껴졌습니다. 그리고 인도의 문화나 음식들을 볼 수 있어서 좋았어요.', 'label': '긍정', 'reason': '결말이 좋았고 인도의 문화와 음식을 볼 수 있어서 좋았다는 표현에서 긍정을 판단했다.'}

=== SFT input sample ===
문제: 저는 오히려 결말이 좋았다고 느꼈는데 보여주진 않았지만 저는 만났을 거라고 생각합니다. 잔잔한 여운과 쓸쓸함이 느껴졌습니다. 그리고 인도의 문화나 음식들을 볼 수 있어서 좋았어요.
정답: 긍정
근거: 결말이 좋았고 인도의 문화와 음식을 볼 수 있어서 좋았다는 표현에서 긍정을 판단했다.


In [ ]:
# 7) SFT 학습 + 성능 확인

import torch
from trl import SFTTrainer
from transformers import TrainingArguments

print("\n=== BEFORE reasoning SFT ===")
_ = test_model(peft_r2, tokenizer_r2, sampled_test, pos_token_id_r2, neg_token_id_r2)

trainer_reason = SFTTrainer(
    model=peft_r2,
    tokenizer=tokenizer_r2,
    train_dataset=train_reason_ds,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=1e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs_r2_reasoning",
        num_train_epochs=1,
        report_to="none",
        group_by_length=False,
    ),
    formatting_func=formatting_prompts_reasoning,
)




=== BEFORE reasoning SFT ===


100%|██████████| 100/100 [00:12<00:00,  8.04it/s]


Correct: 721 Total: 1000 Accuracy: 0.7210


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print("\n=== TRAIN reasoning SFT ===")
trainer_reason.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 63
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 573,440 of 3,213,323,264 (0.02% trained)



=== TRAIN reasoning SFT ===


Step,Training Loss
10,2.754700
20,2.404800
30,2.303400
40,2.184900
50,2.021500
60,2.120500


TrainOutput(global_step=63, training_loss=2.2937850043887185, metrics={'train_runtime': 86.6063, 'train_samples_per_second': 11.546, 'train_steps_per_second': 0.727, 'total_flos': 1.209894619127808e+16, 'train_loss': 2.2937850043887185, 'epoch': 1.0})

In [ ]:
print("\n=== AFTER reasoning SFT ===")
_ = test_model(peft_r2, tokenizer_r2, sampled_test, pos_token_id_r2, neg_token_id_r2)


=== AFTER reasoning SFT ===


100%|██████████| 100/100 [00:12<00:00,  8.12it/s]

Correct: 709 Total: 1000 Accuracy: 0.7090
